In [1]:
import pandas as pd  
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("Apple_data.csv")  
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,2015-01-02 00:00:00-05:00,27.847500,27.860001,26.837500,27.332500,24.320435,212818400,0.0,0.0
1,2015-01-05 00:00:00-05:00,27.072500,27.162500,26.352501,26.562500,23.635286,257142000,0.0,0.0
2,2015-01-06 00:00:00-05:00,26.635000,26.857500,26.157499,26.565001,23.637506,263188400,0.0,0.0
3,2015-01-07 00:00:00-05:00,26.799999,27.049999,26.674999,26.937500,23.968960,160423600,0.0,0.0
4,2015-01-08 00:00:00-05:00,27.307501,28.037500,27.174999,27.972500,24.889904,237458000,0.0,0.0


In [3]:
# Convert 'Date' column to datetime and set it as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [4]:
data = df[['Close']]

In [5]:
split_index = int(len(data) * 0.8)
train = data.iloc[:split_index]
test = data.iloc[split_index:]
print("Train shape:", train.shape)
print("Test shape:", test.shape)

Train shape: (2012, 1)
Test shape: (504, 1)


In [6]:
from statsmodels.tsa.stattools import adfuller

# Run ADF test on training data
result = adfuller(train['Close'])

print("ADF Statistic:", result[0])
print("p-value:", result[1])
for key, value in result[4].items():
    print(f"Critical Value ({key}): {value}")

ADF Statistic: -0.7221179864347376
p-value: 0.8409370017466231
Critical Value (1%): -3.4336386745240652
Critical Value (5%): -2.8629927557359443
Critical Value (10%): -2.5675433856598793


In [7]:
# First-order differencing
train_diff = train['Close'].diff().dropna()

# ADF test on differenced data
result_diff = adfuller(train_diff)

print("ADF Statistic (differenced):", result_diff[0])
print("p-value:", result_diff[1])
for key, value in result_diff[4].items():
    print(f"Critical Value ({key}): {value}")

ADF Statistic (differenced): -9.119995028094099
p-value: 3.224527732398468e-15
Critical Value (1%): -3.4336386745240652
Critical Value (5%): -2.8629927557359443
Critical Value (10%): -2.5675433856598793


In [8]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(train['Close'], order=(3, 1, 1)) 
model_fit = model.fit()

/Users/admin/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/admin/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/admin/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [9]:
print(model_fit.summary())

                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                 2012
Model:                 ARIMA(3, 1, 1)   Log Likelihood               -4007.173
Date:                Wed, 26 Mar 2025   AIC                           8024.346
Time:                        23:59:52   BIC                           8052.377
Sample:                             0   HQIC                          8034.635
                               - 2012                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0268      0.393     -0.068      0.946      -0.796       0.743
ar.L2         -0.0277      0.025     -1.129      0.259      -0.076       0.020
ar.L3         -0.0311      0.017     -1.838      0.0

In [10]:
forecast = model_fit.forecast(steps=len(test))
forecast.index = test.index

from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(test['Close'], forecast)
rmse = np.sqrt(mean_squared_error(test['Close'], forecast))

print("MAE:", mae)
print("RMSE:", rmse)

/Users/admin/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/admin/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


MAE: 63.144135456929085
RMSE: 69.07356494827062
